In [1]:
from twitter.search import Search
from datetime import datetime
import boto3

In [ ]:
proxy_credentials = {"username": "username", "password": "password", "bearer_token": "bearer_token"}

In [ ]:
s3 = boto3.client('s3', aws_access_key_id="aws_acces_key", aws_secret_access_key="aws_secret_acces_key", endpoint_url="end_point_yrl")
twitter_accounts = {"s3": s3, "bucket_name": "bucket_name", "file_name": "twitter/scrapping/example_accounts.json"}

In [ ]:
query = '("americanas" OR "@americanas") lang:pt until:2023-09-25 since:2023-09-24'

In [ ]:
search = Search(twitter_accounts="example_accounts.json", # OR twitter_accounts=twitter_accounts to use S3
                collection_limit_per_account=500,
                hours_to_reset_collection=12,
                proxy_credentials=proxy_credentials,
                debug=True
                )

latest_results = search.run(
    limit=1000,
    latest=True,
    retries=3,
    queries=[
        {
            'category': 'Latest',
            'query': query
        },
    ],
)

df = search.get_tweets_dataframe()

In [ ]:
df.to_excel(f"twitter-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.xlsx")

In [ ]:
df

In [ ]:
import pandas as pd
import os

def read_and_join_dataframes(folder_path: str) -> pd.DataFrame:
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    joined_df = pd.DataFrame()
    for file in all_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        if joined_df.empty:
            joined_df = df
        else:
            joined_df = pd.concat([joined_df, df], ignore_index=True)
    return joined_df

joined_df = read_and_join_dataframes("path_to_folder")